In [1]:
import pandas as pd 
import numpy as np
import os
import glob
from datetime import timedelta

In [2]:
file_path = '/Users/antonebalakov/Downloads/train_data_Sila/data_example/data_Q4_2023/'
#считываем данные
files = ['2023-10-01.csv', '2023-10-02.csv', '2023-10-03.csv', '2023-10-04.csv', '2023-10-05.csv',
         '2023-10-06.csv', '2023-10-07.csv', '2023-10-08.csv', '2023-10-09.csv', '2023-10-10.csv',
        '2023-10-11.csv', '2023-10-12.csv', '2023-10-13.csv', '2023-10-14.csv', '2023-10-15.csv',
        '2023-10-16.csv', '2023-10-17.csv', '2023-10-18.csv', '2023-10-19.csv', '2023-10-20.csv']
data = pd.DataFrame()
i = 0
for filename in files:
    temp = pd.read_csv(file_path+filename)
    temp = temp.drop_duplicates(subset=['date', 'serial_number'])
    data = pd.concat([data, temp])
data

,date,serial_number,model,capacity_bytes,failure,datacenter,cluster_id,vault_id,pod_id,pod_slot_num,...,smart_250_normalized,smart_250_raw,smart_251_normalized,smart_251_raw,smart_252_normalized,smart_252_raw,smart_254_normalized,smart_254_raw,smart_255_normalized,smart_255_raw
0,2023-10-01,WD-WX31DB48X22V,WDC WD60EFRX,6001175126016,0,sac0,0,1002,0,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-10-01,Z4D00WGP,ST6000DX000,6001175126016,0,sac0,0,1002,0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-10-01,Z4D00YC6,ST6000DX000,6001175126016,0,sac0,0,1002,0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-10-01,Z4D00YS2,ST6000DX000,6001175126016,0,sac0,0,1002,0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-10-01,Z4D029JS,ST6000DX000,6001175126016,0,sac0,0,1002,0,38.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268203,2023-10-20,8240A021FVKG,TOSHIBA MG08ACA16TA,16000900661248,0,iad1,52,1006,19,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
268204,2023-10-20,8240A027FVKG,TOSHIBA MG08ACA16TA,16000900661248,0,iad1,52,1006,19,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
268205,2023-10-20,8240A028FVKG,TOSHIBA MG08ACA16TA,16000900661248,0,iad1,52,1006,19,57.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
268206,2023-10-20,8240A029FVKG,TOSHIBA MG08ACA16TA,16000900661248,0,iad1,52,1006,19,41.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#убираем фичи где слишком много пропусков
data['date'] = pd.to_datetime(data['date'])

feature_list = []

for column in data.columns:
    if (data[column].nunique() >= 1) & (data[column].notnull().sum() > 0.01 * data.shape[0]):
        feature_list.append(column)
        
data = data[feature_list]
#считываем данные таргетов, которые произойдут через 30 дней
dates_to_failure =  ['2023-10-31.csv', '2023-11-01.csv', '2023-11-02.csv', '2023-11-03.csv', '2023-11-04.csv', '2023-11-05.csv',
         '2023-11-06.csv', '2023-11-07.csv', '2023-11-08.csv', '2023-11-09.csv', '2023-11-10.csv',
        '2023-11-11.csv', '2023-11-12.csv', '2023-11-13.csv', '2023-11-14.csv', '2023-11-15.csv',
        '2023-11-16.csv', '2023-11-17.csv', '2023-11-18.csv', '2023-11-19.csv', '2023-11-20.csv']

failure_data = pd.DataFrame()
for filename in dates_to_failure:
    temp = pd.read_csv(file_path+filename)
    temp = temp.drop_duplicates(subset=['date', 'serial_number'])[['date', 'serial_number', 'failure']]
    
    failure_data = pd.concat([failure_data, temp])
failure_data

,date,serial_number,failure
0,2023-10-31,WD-WX31DB48X22V,0
1,2023-10-31,Z4D00WGP,0
2,2023-10-31,Z4D00YC6,0
3,2023-10-31,Z4D00YS2,0
4,2023-10-31,Z4D029JS,0
...,...,...,...
272361,2023-11-20,8240A021FVKG,0
272362,2023-11-20,8240A027FVKG,0
272363,2023-11-20,8240A028FVKG,0
272364,2023-11-20,8240A029FVKG,0


In [4]:
#мержим данные таргетов с основными данными 
failure_data['date'] = pd.to_datetime(failure_data['date'])
failure_data['date'] = failure_data['date'] - timedelta(days=30)
failure_data = failure_data.rename(columns={'failure': 'target'})
data = data.merge(failure_data, on=['date', 'serial_number'], how='left')
failure_data

,date,serial_number,target
0,2023-10-01,WD-WX31DB48X22V,0
1,2023-10-01,Z4D00WGP,0
2,2023-10-01,Z4D00YC6,0
3,2023-10-01,Z4D00YS2,0
4,2023-10-01,Z4D029JS,0
...,...,...,...
272361,2023-10-21,8240A021FVKG,0
272362,2023-10-21,8240A027FVKG,0
272363,2023-10-21,8240A028FVKG,0
272364,2023-10-21,8240A029FVKG,0


In [7]:
#сортируй датафрейм (сортирутеся 5-10 минут)
data = data.sort_values(['serial_number', 'date'])

In [6]:
#оставляем только нормализованные фичи и строим по ним EMA (экспоненциальное скользящее среднее)
smart_features = []
for column in data.columns:
    if column[-10:] == 'normalized':
        smart_features.append(column)
base_features = ['date', 'serial_number', 'model', 'capacity_bytes', 'failure', 'datacenter', 'cluster_id',
                 'vault_id', 'pod_id', 'pod_slot_num', 'is_legacy_format', 'target']

data = data[base_features + smart_features]
data = data.sort_values(['serial_number', 'date'])

#Вычисление скользящего среднего
window_size = 7
for feature in smart_features:
    data[feature+'_ema'] = data.groupby('serial_number')[feature].transform(lambda x: x.ewm(span=window_size, adjust=False).mean())
   







In [ ]:
#Вычисление изменений фич за 1 день, 2 дня ... 7 дней (регулируется)

size = 7
for i in range(1, size+1):
    for feature in smart_features:
        data[feature+'_back'] = data.groupby('serial_number')[feature].shift(i)
        data[feature+'_chg'] = (data[feature] / data[feature+'_back']) - 1
        data = data.drop(columns=[feature+'_back'])
        
data    


            Value      EMA
Date                      
2023-01-01    100  100.000
2023-01-02    150  125.000
2023-01-03    120  122.500
2023-01-04    180  151.250
2023-01-05    110  130.625
